In [1]:
push!(LOAD_PATH, joinpath(@__DIR__, "..", "src"))

4-element Vector{String}:
 "@"
 "@v#.#"
 "@stdlib"
 "/net/fs09/d0/jpalmo/Projects/O3Fire/paper/code/boxmodel/figures/../src"

In [3]:
using Plots
using CairoMakie
using StatsPlots
using Statistics
using LaTeXStrings

In [4]:
using CSV
using DataFrames
using CategoricalArrays

melted_data = CSV.read("sensitivity_results.csv", DataFrame);

# Plot

In [5]:
# Define the custom order for the Factor column
custom_order = ["CO", "NOxCO", "VOCCO", "CH2OCO", "HNO2CO", "I", "ASA", "default"]

# Convert the Factor column to a CategoricalArray with the custom order
melted_data[!, :Factor] = CategoricalArray(melted_data[!, :Factor], ordered=true, levels=custom_order)

# Sort the DataFrame by the Factor column
sorted_data = sort(melted_data, :Factor);

In [6]:
# Calculate the mean of Sensitivity within the subset of Factor == "default" for each Case
mean_sensitivity_default = combine(groupby(filter(row -> row.Factor == "default", sorted_data), :Case), :Sensitivity => mean => :Mean_Sensitivity);

In [7]:
# Drop any rows with Factor == "default" from sorted_data
filtered_data = filter(row -> row.Factor != "default", sorted_data);

In [8]:
# Define custom colors for the Case groups
custom_colors = [palette(:tab10)[2], palette(:tab10)[1], palette(:tab10)[3]]  # Specify colors as a vector

In [9]:
# Define the LaTeX labels for the Factor column
latex_labels = [
    L"CO",  
    L"\frac{NO_x}{CO}",  
    L"\frac{VOC}{CO}",  
    L"\frac{CH_2O}{CO}",  
    L"\frac{HONO}{CO}",  
    L"I",  
    L"ASA", 
]

7-element Vector{LaTeXString}:
 L"$CO$"
 L"$\frac{NO_x}{CO}$"
 L"$\frac{VOC}{CO}$"
 L"$\frac{CH_2O}{CO}$"
 L"$\frac{HONO}{CO}$"
 L"$I$"
 L"$ASA$"

In [13]:
plotting_data = filter(row -> row.Case == "far_field", filtered_data)
@df plotting_data StatsPlots.boxplot(:Factor, :Sensitivity, group=:Case, fill = (0.5,), line=0.5, linecolors = custom_colors[1], palette = custom_colors, label="far-field", xlabel="Factor", ylabel="Sensitivity", fmt=:png, legend=:topright, grid=false, size=(800, 250), dpi=300, markersize=2,)
plotting_data = filter(row -> row.Case == "near_field", filtered_data)
@df plotting_data StatsPlots.boxplot!(:Factor, :Sensitivity, group=:Case, fill = (0.5,), line=0.5, linecolors = custom_colors[2], palette = custom_colors, label="near-field", xlabel="Factor", ylabel="Sensitivity", fmt=:png, legend=:topright, grid=false, dpi=300, markersize=2)

# Plot a horizontal line at y=10
hline!(mean_sensitivity_default[mean_sensitivity_default.Case .== "near_field", :Mean_Sensitivity], line=:dash, color=custom_colors[2], label="")
# StatsPlots.ylims!(-1, 25)
p = StatsPlots.plot!(legendfontsize=16, tickfontsize=12, guidefontsize=14, titlefontsize=16, xlabelfontsize=14, ylabelfontsize=14, left_margin = 6Plots.mm, bottom_margin = 10Plots.mm)
hline!(mean_sensitivity_default[mean_sensitivity_default.Case .== "far_field", :Mean_Sensitivity], line=:dash, color=custom_colors[1], label="")
Plots.xlabel!("Factor")
# Set the x-axis labels to LaTeX strings
Plots.xticks!(1:length(latex_labels), latex_labels)
Plots.yticks!([10, 20, 30], ["10", "20", "30"])
Plots.ylabel!(L"$P_{O_3}$ [ppb/hr]")
# Save the figure
savefig(p, "../../../figures/fig_07c2.png",)

"/net/fs09/d0/jpalmo/Projects/O3Fire/paper/figures/fig_07c2.png"

In [12]:
plotting_data = filter(row -> row.Case == "toe", filtered_data)
@df plotting_data StatsPlots.boxplot(:Factor, :Sensitivity, group=:Case, fill = (0.7,), linecolors = palette(:tab10)[3], palette = [palette(:tab10)[3]], line = 0.5, label="at emission", xlabel="Factor", ylabel="Sensitivity", fmt=:png, legend=:topleft, grid=false, xticks=false, size=(800, 175), dpi=300, markersize=2)
# StatsPlots.ylims!(-1, 25)
hline!(mean_sensitivity_default[mean_sensitivity_default.Case .== "toe", :Mean_Sensitivity], line=:dash, color=custom_colors[3], label="")
p = StatsPlots.plot!(legendfontsize=16, tickfontsize=12, guidefontsize=14, titlefontsize=16, xlabelfontsize=14, ylabelfontsize=14, left_margin = 6Plots.mm, bottom_margin = -1Plots.mm)
Plots.xlabel!("Factor")
Plots.yticks!([100, 200, 300], ["100", "200", "300"])
Plots.ylabel!(L"$P_{O_3}$ [ppb/hr]")
savefig(p, "../../../figures/fig_07c1.png",)

"/net/fs09/d0/jpalmo/Projects/O3Fire/paper/figures/fig_07c1.png"